In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from EDA_script_original_amy import *
import lightgbm as lgb

In [4]:
X_train, X_val, y_train, y_val = train_test_split(df_train, df_y, test_size=0.3, random_state=42)

# Define a range of values for the hyperparameters to search through
# param_grid = {
#     'num_leaves': int(hp.quniform('num_leaves', 0 , 40 , 1)),
#     'learning_rate': float(hp.uniform('learning_rate', 0.01, 0.3)),
#     'n_estimators': int(hp.quniform('n_estimators', 0,1)),
#     'early_stopping_rounds': 10,
#     'max_depth': int(hp.quniform("max_depth", 3, 18, 1)),
#     'lambda_l1': float(hp.uniform('lambda_l1', 0,1)),
#     'lambda_l2': float(hp.uniform('lambda_l2', 0,1)),
#     'min_data_in_leaf':  int(hp.quniform('min_data_in_leaf', 0 , 40 , 1)),
#     'min_gain_to_split': float(hp.uniform('min_gain_to_split', 0.005, 0.35)),
#     'feature_fraction': float(hp.uniform('feature_fraction', 0,1)),
#     'bagging_fraction': float(hp.uniform('bagging_fraction', 0,1))
# }

param_grid = {
    'num_leaves': [31, 40, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'lambda_l1': [0, 0.5, 1],
    'lambda_l2': [0, 0.5, 1],
    'min_child_samples': [20, 30, 40],  # Corresponds to min_data_in_leaf
}

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Apply'

In [ ]:
# Initialize an empty dictionary to store the best hyperparameters and their values
best_hyperparameters = {}
best_values = {}
 
# Initialize the LightGBM classifier
lgb_classifier = lgb.LGBMClassifier(objective='multiclass', num_class=5, boosting_type='gbdt', metric = 'multi_logloss', 
                                    early_stopping_rounds='early_stopping_rounds', verbosity = -1)
 
# Initialize GridSearchCV for hyperparameters
grid_search = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid,
                           scoring='accuracy', cv=5)
 
# Fit the model to the training data to search for the best hyperparameters
grid_search.fit(X_train, y_train)
 
# Get the best hyperparameters and their values
best_params = grid_search.best_params_
best_hyperparameters = list(best_params.keys())
best_values = list(best_params.values())

In [ ]:
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)

In [ ]:
y_pred = best_model.predict(X_test)
 
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
 
print('Best hyperparameters:', best_hyperparameters)
print('Best values:', best_values)
print(f'Accuracy with best hyperparameters: {accuracy:.4f}')